In [25]:
import pandas as pd
from googleapiclient.discovery import build

file_path = 'Climate_change_videos_data.xlsx' 
df = pd.read_excel(file_path)

df['Link'] = df['Link'].astype(str)

video_links = df['Link']

# get video ID
def get_video_id(url):
    if "youtube.com" in url:
        return url.split('v=')[-1]
    elif "youtu.be" in url:
        return url.split('/')[-1]
    return None

api_key = 'YOUTUBE_API'  # 请替换为你的实际API密钥
youtube = build('youtube', 'v3', developerKey=api_key)

# get license
def get_video_licenses(video_ids):
    licenses = {}
    response = youtube.videos().list(
        part='status',
        id=','.join(video_ids)
    ).execute()

    for item in response['items']:
        video_id = item['id']
        license_type = item['status']['license']
        licenses[video_id] = license_type
    return licenses

video_ids = [get_video_id(url) for url in video_links if get_video_id(url)]

all_licenses = get_video_licenses(video_ids)

# add a column in original file
df['License'] = df['Link'].apply(lambda x: all_licenses.get(get_video_id(x), 'Unknown'))

df_first_20 = df[:20]

output_file_path = 'with_license.xlsx' 
df.to_excel(output_file_path, index=False)

print("许可信息已成功添加并保存到新的Excel文件。")


许可信息已成功添加并保存到新的Excel文件。
